In [ ]:
import numpy as np
import os
import shutil
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time

# Original Settings

In [ ]:
elems0,lattices,labor = mli.readInputfile('mli.in.origin')

In [ ]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [ ]:
elems = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elems.append(item)

In [ ]:
labor = ['iotaline','mapout','fin']
mli.writeInputfile(elems,lattices,labor)
mli.run()
M,G = mli.readTransferMap()

### Map

In [ ]:
M,G = mli.readTransferMap()
M

In [ ]:
G.head()

### Investigate Quadrupole settings

In [ ]:
for i,item in enumerate(elems):
    if 'quadrupole' == item.elem:
        for k,v in item.items():
            if k not in ['name','elem','l']:
                print(k,v)

In [ ]:
indexQuad = []
QuadStrength = []
for i,item in enumerate(elems):
    if 'quadrupole' == item.elem:
        if 'k1' in item.keys():
            indexQuad.append(i)
            QuadStrength.append(item.k1)
nQuad = len(indexQuad)

In [ ]:
for i in range(len(QuadStrength)):
    print(QuadStrength[i],QuadStrength[-i-1],QuadStrength[i]-QuadStrength[-i-1])

### Sextupoles

In [ ]:
indexSext = []
SextStrength = []
for i,item in enumerate(elems):
    if 'sextupole' == item.elem:
        indexSext.append(i)
        SextStrength.append(item.g2)
nSext = len(indexSext)

# Optimize

In [ ]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx

In [ ]:
M_ref = np.array([[ 1.0, 0.0,  0.0,  0.0,  0.0],
                  [ -k , 1.0,  0.0,  0.0,  0.0],
                  [ 0.0, 0.0,  1.0,  0.0,  0.0],
                  [ 0.0, 0.0,   -k,  1.0,  0.0]])

In [ ]:
M.values[:4,[0,1,2,3,5]]-M_ref

In [ ]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0
G2_ref

In [ ]:
G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.head()

In [ ]:
G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0
G4_ref.head()

In [ ]:
G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0

In [ ]:
print(len(G2_ref),len(G3_ref),len(G4_ref),len(G5_ref))

In [ ]:
#%%
def objFunc(arg): 
    target = opt.id_generator()  # generage random directory name
    while os.path.exists(target):  
        target = opt.id_generator()
    shutil.copytree('origin', target)
    time.sleep(0.1)
    os.chdir(target) # cd to the randome directory and
    
    for i,j in enumerate(indexQuad):
        elems[j]['k1']=arg[i]
    for i,j in enumerate(indexSext):
        elems[j]['g2']=arg[i+nQuad]
        
    mli.writeInputfile(elems,lattices,labor)
    mli.run()
    time.sleep(0.1)
    M,G = mli.readTransferMap()
    
    obj = 1.0e36* np.sum((M.values[:4,[0,1,2,3,5]]-M_ref)**2) \
         +1.0e30* np.sum((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)**2) \
         +1.0e22* np.sum((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)**2) \
         +1.0e12* np.sum((G.loc[210:450,'GP'].values-G4_ref['GP'].values)**2) \
         +        np.sum((G.loc[462:910,'GP'].values-G5_ref['GP'].values)**2)
    time.sleep(0.1)
    os.chdir('..')
    time.sleep(0.1)
    shutil.rmtree(target)
    time.sleep(0.1)
    return obj

In [ ]:
objFunc(QuadStrength+[0]*nSext)

In [ ]:
#%% run optim
bounds = []
for k1 in QuadStrength:
    bounds.append((k1-0.02*np.abs(k1),k1+0.02*np.abs(k1)))
for g2 in SextStrength:
    bounds.append((-200.0,200.0))
print(bounds)    
# result=opt.differential_evolution(objFunc, bounds, ncore=32, popsize=32*4,
#                                         disp=True, polish=True, maxtime=60*10) 
#                                         # stop running at maximum 10 min

In [ ]:
# import pickle
# with open('result.quad.sext','wb') as fp:
#     pickle.dump(result,fp)

In [ ]:
import pickle
with open('result.quad.sext','rb') as fp:
    result=pickle.load(fp)
result.population_energies[0]

In [ ]:
while True:
    previous_result = result
    if hasattr(result,'x'): 
        break
    result = opt.differential_evolution(objFunc, bounds, ncore=32, 
                                           prev_result=previous_result, 
                                           disp=True, polish=True, maxtime=60*10)
    with open('result.quad.sext','wb') as fp:
        pickle.dump(result,fp)
        time.sleep(0.1)